## Ordering Basemaps via Orders API

In [12]:
import planet
import os
import requests
from requests.auth import HTTPBasicAuth
import json
import time
import pathlib

### Ordering a Basemap via an AOI  

step 1: find name of basemap on:
    planet explorer
    basemap api URL


step 2: create order

In [15]:
API_KEY = 'api key'
orders_url = 'https://api.planet.com/compute/ops/orders/v2' 
auth = HTTPBasicAuth('api key', '')
if os.environ.get(API_KEY, ''):
    API_KEY = os.environ.get(API_KEY, '')
else:
    API_KEY = API_KEY
    
# Setup the session
session = requests.Session()

# Authenticate
session.auth = (API_KEY, "")

In [16]:
response = requests.get(orders_url, auth=session.auth)
response

<Response [200]>

In [16]:
order_params = {
   "name": "basemap order with geometry",
   "source_type": "basemaps",
   "order_type":"partial",
   "products": [
       {
           "mosaic_name": "global_monthly_2022_01_mosaic",
           "geometry":{
               "type": "Polygon",
               "coordinates":[
                   [
                       [ 4.607406, 52.353994 ],
                        [ 4.680005, 52.353994 ],
                        [ 4.680005, 52.395523 ],
                        [ 4.607406, 52.395523 ],
                        [ 4.607406, 52.353994 ]
                   ]
               ]
           }
      }
  ],
    "tools": [
      {"merge": {}},
      {"clip": {}}
      
  ],
 "delivery":{
      "google_cloud_storage":{
          "bucket":"[your-orders-delivery-bucket]",
   #       "credentials":"[QUl6YVN5QWNlSzZFV1pGZ2I3Yklfc285UzNVTDRDanZfYlB1SDdVCg==]",
   #       "path_prefix":"[optionalsubfolder/]"
      }


 }
}

In [18]:
ORDERS_API_URL = 'https://api.planet.com/compute/ops/orders/v2'
headers = {'content-type': 'application/json'}
paramRes = requests.post(ORDERS_API_URL, data=json.dumps(order_params), auth=auth, headers = {'content-type': 'application/json'})

print(paramRes.text)

{"_links":{"_self":"https://api.planet.com/compute/ops/orders/v2/bda6aaae-b2ff-48f9-a794-0d2c4342a9ea"},"created_on":"2022-11-18T22:39:28.111Z","delivery":{"google_cloud_storage":{"bucket":"notebook-practice","credentials":"<REDACTED>","path_prefix":"basemaps-to-cloud/"}},"error_hints":[],"id":"bda6aaae-b2ff-48f9-a794-0d2c4342a9ea","last_message":"Preparing order","last_modified":"2022-11-18T22:39:28.111Z","name":"basemap order with geometry","order_type":"partial","products":[{"geometry":{"coordinates":[[[4.607406,52.353994],[4.680005,52.353994],[4.680005,52.395523],[4.607406,52.395523],[4.607406,52.353994]]],"type":"Polygon"},"mosaic_name":"global_monthly_2022_01_mosaic"}],"source_type":"basemaps","state":"queued","tools":[{"merge":{}},{"clip":{}}]}



In [19]:
def place_order(order_params, auth):
    response = requests.post(ORDERS_API_URL, data=json.dumps(order_params), auth=auth, headers=headers)
    print(response)
    order_id = response.json()['id']
    print(order_id)
    order_url = ORDERS_API_URL + '/' + order_id
    return order_url

In [20]:
order_url = place_order(order_params, session.auth)

<Response [202]>
36aa44c6-56ed-4c78-8570-105bebc13a6a


In [21]:
def poll_for_success(order_url, auth, num_loops=30):
    count = 0
    while(count < num_loops):
        count += 1
        r = requests.get(order_url, auth=session.auth)
        response = r.json()
        print(response)
        
        state = response['state']
        print(state)
        end_states = ['success', 'failed', 'partial']
        if state in end_states:
            break
        time.sleep(10)
        # print(response)
        
poll_for_success(order_url, auth)

{'_links': {'_self': 'https://api.planet.com/compute/ops/orders/v2/36aa44c6-56ed-4c78-8570-105bebc13a6a'}, 'created_on': '2022-11-18T22:39:28.810Z', 'delivery': {'google_cloud_storage': {'bucket': 'notebook-practice', 'credentials': '<REDACTED>', 'path_prefix': 'basemaps-to-cloud/'}}, 'error_hints': [], 'id': '36aa44c6-56ed-4c78-8570-105bebc13a6a', 'last_message': 'Preparing order', 'last_modified': '2022-11-18T22:39:28.810Z', 'name': 'basemap order with geometry', 'order_type': 'partial', 'products': [{'geometry': {'coordinates': [[[4.607406, 52.353994], [4.680005, 52.353994], [4.680005, 52.395523], [4.607406, 52.395523], [4.607406, 52.353994]]], 'type': 'Polygon'}, 'mosaic_name': 'global_monthly_2022_01_mosaic'}], 'source_type': 'basemaps', 'state': 'queued', 'tools': [{'merge': {}}, {'clip': {}}]}
queued
{'_links': {'_self': 'https://api.planet.com/compute/ops/orders/v2/36aa44c6-56ed-4c78-8570-105bebc13a6a'}, 'created_on': '2022-11-18T22:39:28.810Z', 'delivery': {'google_cloud_stora

In [22]:
r = requests.get(order_url, auth=session.auth)
response = r.json()
results = response['_links']['results']

KeyError: 'results'

In [ ]:
[r['name'] for r in results]

['basemaps-to-cloud/9e15388e-6c11-4945-b69c-3733e2e8d05d/global_monthly_2022_01_mosaic/global_monthly_2022_01_mosaic_metadata_merge_clip.json',
 'basemaps-to-cloud/9e15388e-6c11-4945-b69c-3733e2e8d05d/global_monthly_2022_01_mosaic/global_monthly_2022_01_mosaic_provenance_vector_merge_clip.zip',
 'basemaps-to-cloud/9e15388e-6c11-4945-b69c-3733e2e8d05d/global_monthly_2022_01_mosaic/global_monthly_2022_01_mosaic_ortho_udm_merge_clip.tif',
 'basemaps-to-cloud/9e15388e-6c11-4945-b69c-3733e2e8d05d/global_monthly_2022_01_mosaic/global_monthly_2022_01_mosaic_merge_clip.tif',
 'basemaps-to-cloud/9e15388e-6c11-4945-b69c-3733e2e8d05d/global_monthly_2022_01_mosaic/global_monthly_2022_01_mosaic_provenance_raster_merge_clip.tif',
 'basemaps-to-cloud/9e15388e-6c11-4945-b69c-3733e2e8d05d/manifest.json']

In [ ]:
def download_results(results, overwrite=False):
    results_urls = [r['location'] for r in results]
    results_names = [r['name'] for r in results]
    print('{} items to download'.format(len(results_urls)))
    
    for url, name in zip(results_urls, results_names):
        path = pathlib.Path(os.path.join('data', name))
        
        if overwrite or not path.exists():
            print('downloading {} to {}'.format(name, path))
            r = requests.get(url, allow_redirects=True)
            path.parent.mkdir(parents=True, exist_ok=True)
            open(path, 'wb').write(r.content)
        else:
            print('{} already exists, skipping {}'.format(path, name))
download_results(results)

KeyError: 'location'

### Ordering a Basemap With a Set of Quad IDs

you can get the quad IDs from planet explorer or if you have the basemap, you can find the quad IDs like this:

In [1]:
ORDERS_API_URL = 'https://api.planet.com/compute/ops/orders/v2'

In [2]:
API_KEY = 'PLAK0962efb566a34514a56599994d2d0674'
orders_url = 'https://api.planet.com/compute/ops/orders/v2' 
auth = HTTPBasicAuth('PLAK0962efb566a34514a56599994d2d0674', '')
if os.environ.get(API_KEY, ''):
    API_KEY = os.environ.get(API_KEY, '')
else:
    API_KEY = API_KEY
    
# Setup the session
session = requests.Session()

# Authenticate
session.auth = (API_KEY, "")

NameError: name 'HTTPBasicAuth' is not defined

In [3]:
response = requests.get(orders_url, auth=session.auth)
response

NameError: name 'requests' is not defined

basemap_params = {
    'name__is': 'point_reyes_rolling_normalized_2020-04-30_mosaic'
}

In [4]:
auth = HTTPBasicAuth('PLAK0962efb566a34514a56599994d2d0674', '')
BASEMAP_API_URL = 'https://api.planet.com/basemaps/v1/mosaics'
headers = {'content-type': 'application/json'}

basemap_params = {
    'name__is': 'point_reyes_rolling_normalized_2020-04-30_mosaic'
}

basemapServiceResponse = requests.get(url=BASEMAP_API_URL, params=basemap_params, auth=auth, headers=headers)

basemaps = basemapServiceResponse.raise_for_status()
if basemapServiceResponse.status_code != 204:
    basemaps = json.loads(basemapServiceResponse.text)
print(basemaps)

NameError: name 'HTTPBasicAuth' is not defined

In [5]:
mosaicID = basemaps['mosaics'][0]['id']
mosaicBbox = basemaps['mosaics'][0]['bbox']
mosaicBboxStr = ','.join(map(str, mosaicBbox))

quad_params = {
    'mosaic_id': mosaicID,
    'bbox': mosaicBboxStr,
}

NameError: name 'basemaps' is not defined

In [6]:
quads_url = "{}/{}/quads".format(BASEMAP_API_URL, mosaicID)
quadServiceResponse = requests.get(url=quads_url, params=quad_params, auth=auth, headers=headers)
quads = quadServiceResponse.json()
items = quads['items']

NameError: name 'BASEMAP_API_URL' is not defined

In [7]:
quadIds = []
for i in items:
    quadId = i['id']
    quadIds.append(quadId)

NameError: name 'items' is not defined

In [8]:
order_params = {
   "name":"basemap order with quad_ids",
   "source_type":"basemaps",
   "products":[
      {
         "mosaic_name":"point_reyes_rolling_normalized_2020-04-30_mosaic",
         "quad_ids": quadIds
      }
   ],
    "tools": [
      {
         "reproject": {
        "projection": "EPSG:4326",
        "resolution": 3.0 / 111000.0,
        "kernel": "cubic"
      }
      },
      {
      "bandmath": {
        "b1": "b1",
        "b2": "b2",
        "b3": "b3",
        "b4": "arctan(b1)",
        "b5": "(b4-b3)/(b4+b3)",
        "pixel_type": "32R"
      }
      
      }
   
    ]

    
   # "delivery":{
   #    "google_cloud_storage":{
   #       "bucket":"[your-orders-delivery-bucket]",
   #       "credentials":"[QUl6YVN5QWNlSzZFV1pGZ2I3Yklfc285UzNVTDRDanZfYlB1SDdVCg==]",
   #       "path_prefix":"[optionalsubfolder/]"
   #    }
   # }
}

In [9]:
paramRes = requests.post(ORDERS_API_URL, data=json.dumps(order_params), auth=auth, headers=headers)

paramRes.raise_for_status()
if paramRes.status_code != 204:
    print(paramRes.text)

NameError: name 'requests' is not defined

In [10]:
def place_order(order_params, auth):
    response = requests.post(ORDERS_API_URL, data=json.dumps(order_params), auth=auth, headers=headers)
    print(response)
    order_id = response.json()['id']
    print(order_id)
    order_url = ORDERS_API_URL + '/' + order_id
    return order_url

In [11]:
order_url = place_order(order_params, session.auth)

NameError: name 'session' is not defined

In [12]:
def poll_for_success(order_url, auth, num_loops=30):
    count = 0
    while(count < num_loops):
        count += 1
        r = requests.get(order_url, auth=session.auth)
        response = r.json()
        # print(response)
        state = response['state']
        print(state)
        end_states = ['success', 'failed', 'partial']
        if state in end_states:
            break
        time.sleep(10)
        # print(response)
        
poll_for_success(order_url, auth)

NameError: name 'order_url' is not defined

In [13]:
r = requests.get(order_url, auth=session.auth)
response = r.json()
results = response['_links']['results']

NameError: name 'requests' is not defined

In [14]:
[r['name'] for r in results]

NameError: name 'results' is not defined

In [15]:
def download_results(results, overwrite=False):
    results_urls = [r['location'] for r in results]
    results_names = [r['name'] for r in results]
    print('{} items to download'.format(len(results_urls)))
    
    for url, name in zip(results_urls, results_names):
        path = pathlib.Path(os.path.join('data', name))
        
        if overwrite or not path.exists():
            print('downloading {} to {}'.format(name, path))
            r = requests.get(url, allow_redirects=True)
            path.parent.mkdir(parents=True, exist_ok=True)
            open(path, 'wb').write(r.content)
        else:
            print('{} already exists, skipping {}'.format(path, name))
download_results(results)

NameError: name 'results' is not defined